# Computer Vision III: Detection, Segmentation and Tracking (CV3DST)

- develop an extension of the ReID-based tracker that will make it more robust to occlusions by allowing it to recover from missed detections. 
- Adapt the track management scheme of our ReIDTracker allow it to recover from missed detections.


#### Install and import Python libraries

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
import os
import sys

root_dir = ".."
sys.path.append(os.path.join(root_dir, "src"))


In [23]:
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm.autonotebook import tqdm

import torch
from torch.utils.data import DataLoader
from mot.models.object_detector import FRCNN_FPN
from mot.data.data_track import MOT16Sequences
from mot.tracker.tracker_offline_det import ReIDTrackerOfflineDet
from mot.utils import  cosine_distance
from mot.eval import run_tracker
from scipy.optimize import linear_sum_assignment as linear_assignment
import os.path as osp
from market.models import build_model
from mot.tracker.base import BaseReIDTracker
from mot.eval import run_tracker_raw_seq
import motmetrics as mm

mm.lap.default_solver = "lap"


In [15]:
!nvidia-smi

Mon Oct 31 17:49:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.26       Driver Version: 430.26       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:21:00.0 Off |                  N/A |
| 27%   40C    P8    28W / 260W |   2510MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:41:00.0 Off |                  N/A |
| 27%   34C    P8     1W / 250W |   1681MiB / 11018MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [16]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [17]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


In [18]:
obj_detect_model_file = os.path.join(root_dir, "models/faster_rcnn_fpn.model")
obj_detect_nms_thresh = 0.3

# object detector
obj_detect = FRCNN_FPN(num_classes=2, nms_thresh=obj_detect_nms_thresh)
obj_detect_state_dict = torch.load(
    obj_detect_model_file, map_location=lambda storage, loc: storage
)
obj_detect.load_state_dict(obj_detect_state_dict)
obj_detect.eval()
obj_detect = obj_detect.to(device)


In [34]:
model_path = root_dir + "/models/resnet50_reid.pth"

reid_model = build_model("resnet34", 751, loss="softmax", pretrained=True)
reid_ckpt = torch.load(model_path)
reid_model.load_state_dict(reid_ckpt)
reid_model = reid_model.to(device)

In [20]:
seed = 1234
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True


## ReIDHungarianTracker

our baseline is ReIDTracker.

This tracker works by performing frame-to-frame bipartite matching between newly detected boxes and past tracks based on ReID distance. Whenever a past track cannot be matched, its killed. And whenever, a newly detected box cannot be match, it starts a new trajectory.

We have modified the ``compute_distance`` function in ``data_association`` to include a thresshold on ReID distance (if ReID distance >0.1, matching is not possible). This is important to prevent our tracker from reusing tracks for very dissimilar objects.

Results:
```
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP
MOT16-02 40.5% 57.5% 31.3% 52.2% 96.1%  62  12  38 12  390  8873 203  210 49.1% 0.090
MOT16-05 54.4% 64.4% 47.1% 68.8% 94.0% 133  54  67 12  305  2156 330  149 59.7% 0.142
MOT16-09 49.9% 61.7% 41.9% 66.3% 97.7%  26  13  12  1   82  1793  77   76 63.3% 0.082
MOT16-11 61.1% 67.4% 55.9% 80.2% 96.6%  75  44  24  7  266  1871 176   91 75.5% 0.083
OVERALL  49.6% 62.3% 41.2% 63.5% 96.1% 296 123 141 32 1043 14693 786  526 59.0% 0.097
```


In [29]:
val_sequences = MOT16Sequences(
    "MOT16-reid", root_dir=osp.join(root_dir, "data/MOT16"), vis_threshold=0.0
)


In [97]:

class ReIDHungarianIoUTracker(BaseReIDTracker):
    def __init__(self, obj_detect, reid_model, **kwargs):
        super().__init__(obj_detect=obj_detect, **kwargs)
        self.reid_model = reid_model
        self.tracks = []
        self.track_num = 0
        self.im_index = 0
        self.results = {}
        self.mot_accum = None
        self._UNMATCHED_COST = 255.0

    def data_association(self, boxes, scores, frame):
        crops = self.get_crop_from_boxes(boxes, frame)
        pred_features = self.compute_reid_features(self.reid_model, crops).cpu().clone()

        if self.tracks:
            # track_ids = [t.id for t in self.tracks] # not needed
            track_boxes = torch.stack([t.box for t in self.tracks], axis=0)
            track_features = torch.stack([t.get_feature() for t in self.tracks], axis=0)

            # This will use your similarity measure. Please use cosine_distance!
            distance = self.compute_distance_matrix(
                track_features,
                pred_features,
                track_boxes,
                boxes,
                metric_fn=cosine_distance,
            )

            # Perform Hungarian matching.
            row_idx, col_idx = linear_assignment(distance)
            self.update_tracks(row_idx, col_idx, distance, boxes, scores, pred_features)

        else:
            # No tracks exist.
            self.add(boxes, scores, pred_features)

    def update_tracks(self, row_idx, col_idx, distance, boxes, scores, pred_features):
        # row_idx and col_idx are indices into track_boxes and boxes.
        # row_idx[i] and col_idx[i] define a match.
        # distance[row_idx[i], col_idx[i]] define the cost for that matching.
        track_ids = [t.id for t in self.tracks]

        remove_track_ids = []
        seen_track_ids = []
        seen_box_idx = []
        # Update existing tracks and remove unmatched tracks.

        for track_idx, box_idx in zip(row_idx, col_idx):
            costs = distance[track_idx, box_idx]
            internal_track_id = track_ids[track_idx]
            seen_track_ids.append(internal_track_id)
            # If the costs are equal to _UNMATCHED_COST, it's not a match.
            if costs == self._UNMATCHED_COST:
                remove_track_ids.append(internal_track_id)
            else:
                self.tracks[track_idx].box = boxes[box_idx]
                self.tracks[track_idx].add_feature(pred_features[box_idx])
                seen_box_idx.append(box_idx)

        unseen_track_ids = set(track_ids) - set(seen_track_ids)
        remove_track_ids.extend(list(unseen_track_ids))
        self.tracks = [t for t in self.tracks if t.id not in remove_track_ids]

        # update the feature of a track by using add_feature:
        # self.tracks[my_track_id].add_feature(pred_features[my_feat_index])
        # use the mean feature from the last 10 frames for ReID.

        new_boxes_idx = set(range(len(boxes))) - set(seen_box_idx)
        new_boxes = [boxes[i] for i in new_boxes_idx]
        new_scores = [scores[i] for i in new_boxes_idx]
        new_features = [pred_features[i] for i in new_boxes_idx]
        self.add(new_boxes, new_scores, new_features)

In [102]:
tracker = ReIDHungarianIoUTracker(reid_model=reid_model, obj_detect = obj_detect)


In [105]:
#tracker = ReIDHungarianTrackerOfflineDet(None)
res_seq = run_tracker_raw_seq(sequences=val_sequences, tracker=tracker)

Tracking: MOT16-02


100%|█████████████████████████████████████████| 600/600 [01:53<00:00,  5.29it/s]


Tracks found: 250
Runtime for MOT16-02: 113.8 s.
Tracking: MOT16-05


100%|█████████████████████████████████████████| 837/837 [01:22<00:00, 10.20it/s]


Tracks found: 275
Runtime for MOT16-05: 82.4 s.
Tracking: MOT16-09


100%|█████████████████████████████████████████| 525/525 [01:07<00:00,  7.74it/s]


Tracks found: 82
Runtime for MOT16-09: 68.0 s.
Tracking: MOT16-11


100%|█████████████████████████████████████████| 900/900 [02:07<00:00,  7.04it/s]


Tracks found: 172
Runtime for MOT16-11: 128.3 s.
Runtime for all sequences: 392.4 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP IDt IDa IDm
MOT16-02 38.8% 55.1% 29.9% 52.3% 96.2%  62  11  38 13  383  8870 260  215 48.8% 0.096 119 146  11
MOT16-05 55.7% 66.0% 48.2% 68.8% 94.2% 133  56  65 12  295  2158 185  150 61.9% 0.142  69 138  24
MOT16-09 50.2% 62.0% 42.1% 66.4% 97.8%  26  13  12  1   80  1789  76   78 63.5% 0.083  23  58   5
MOT16-11 62.8% 69.2% 57.5% 80.2% 96.6%  75  42  26  7  266  1868  87   86 76.5% 0.083  25  68   9
OVERALL  49.5% 62.3% 41.1% 63.5% 96.2% 296 122 141 33 1024 14685 608  529 59.5% 0.099 236 410  49


{'MOT16-02': {0: {0: array([584.55145   , 445.04907   , 669.4421    , 704.5543    ,
            0.99999714], dtype=float32),
   1: array([585.1025    , 446.10538   , 669.4914    , 703.70215   ,
            0.99999714], dtype=float32),
   2: array([585.8408    , 445.89444   , 669.9229    , 704.1525    ,
            0.99999714], dtype=float32),
   3: array([585.5758    , 446.3744    , 669.5082    , 704.1702    ,
            0.99999714], dtype=float32),
   4: array([585.77356   , 446.47705   , 669.10077   , 704.4682    ,
            0.99999714], dtype=float32),
   5: array([585.10284   , 445.9779    , 670.0592    , 704.5802    ,
            0.99999714], dtype=float32),
   6: array([584.8802    , 446.228     , 670.5882    , 704.2709    ,
            0.99999714], dtype=float32),
   7: array([584.88257   , 445.98904   , 671.2655    , 704.14124   ,
            0.99999714], dtype=float32),
   8: array([584.36304   , 445.47064   , 670.7125    , 705.33185   ,
            0.99999714], dtype=float

## Long-Term ReID Tracker


The tracker above has an obvious limitation: whenever a track cannot be matched with the detections of a given frame the track will be killed. This means that if our detector misses an object in a single frame (due to e.g. occlusion), we will not be able to recover that track, and we will start a new one. 

To fix this issue, we would like to allow our tracker to maintain tracks that are not matched during data association. We will refer to these tracks as **inactive**. During data association, we will try to match the detected boxes for the current frame to both tracks that are active (i.e. tracks that we were able to match in the previous frame) as well as those that are inactive. Therefore, if a detector misses an object in a frame and the object reappears after a few frames, we will still be able to match it to its corresponding track, instead of creating a new one.

In order to adapt our tracker to have this behavior, we will use the `inactive` attribute from the `track` class (see `tracker/tracker.py`. This attribute will be assigned an integer indicating for how many frames a track has remained unmatched. Whenever we are able to match the track `t`, we will set `t.inactive=0` and, naturally, when tracks are initialized, the class constructor sets `inactive=0`. 

Your job is to maintain the `inactive` attribute of all tracks being kept by tracker so that its value represents the number of frames for which the track has been unmatched. Additionally, we introduce a `patience` parameter. Whenever a track has been inactive for more than `inactive` frames. it will need to be killed.

Results should approximately be around:

```
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP
MOT16-02 45.9% 65.1% 35.4% 52.2% 96.1%  62  12  37 13  390  8873 130  210 49.4% 0.090
MOT16-05 63.4% 75.0% 54.9% 68.8% 94.0% 133  54  67 12  305  2156 283  149 60.3% 0.142
MOT16-09 52.5% 64.9% 44.1% 66.3% 97.7%  26  13  12  1   82  1793  49   76 63.9% 0.083
MOT16-11 68.3% 75.3% 62.5% 80.2% 96.6%  75  44  24  7  266  1871 136   90 75.9% 0.083
OVERALL  55.7% 70.0% 46.2% 63.5% 96.1% 296 123 140 33 1043 14693 598  525 59.4% 0.097
```


In [106]:
class LongTermReIDHungarianTracker(ReIDHungarianIoUTracker):
    def __init__(self,obj_detect, reid_model, patience, *args, **kwargs):
        """Add a patience parameter"""
        super().__init__(obj_detect = obj_detect, reid_model = reid_model,  **kwargs)
        self.patience = patience
        

    def update_results(self):
        """Only store boxes for tracks that are active"""
        for t in self.tracks:
            if t.id not in self.results.keys():
                self.results[t.id] = {}
            if t.inactive == 0:  # Only change
                self.results[t.id][self.im_index] = np.concatenate(
                    [t.box.cpu().numpy(), np.array([t.score])]
                )

        self.im_index += 1

    def update_tracks(self, row_idx, col_idx, distance, boxes, scores, pred_features):
        track_ids = [t.id for t in self.tracks]

        unmatched_track_ids = []
        seen_track_ids = []
        seen_box_idx = []
        for track_idx, box_idx in zip(row_idx, col_idx):
            costs = distance[track_idx, box_idx]
            internal_track_id = track_ids[track_idx]
            seen_track_ids.append(internal_track_id)
            if costs == _UNMATCHED_COST:
                unmatched_track_ids.append(internal_track_id)

            else:
                self.tracks[track_idx].box = boxes[box_idx]
                self.tracks[track_idx].add_feature(pred_features[box_idx])

                # Note: the track is matched, therefore, inactive is set to 0
                self.tracks[track_idx].inactive = 0
                seen_box_idx.append(box_idx)

        unseen_track_ids = set(track_ids) - set(seen_track_ids)
        unmatched_track_ids.extend(list(unseen_track_ids))

        # Update the `inactive` attribute for those tracks that have been
        # not been matched. kill those for which the inactive parameter
        # is > self.patience

        active_tracks = []
        for t in self.tracks:
            if t.id not in unmatched_track_ids:
                active_tracks.append(t)  
            elif t.inactive < self.patience:
                active_tracks.append(t)
                t.inactive += 1
            else:  #
                continue
        self.tracks = active_tracks

        new_boxes_idx = set(range(len(boxes))) - set(seen_box_idx)
        new_boxes = [boxes[i] for i in new_boxes_idx]
        new_scores = [scores[i] for i in new_boxes_idx]
        new_features = [pred_features[i] for i in new_boxes_idx]
        self.add(new_boxes, new_scores, new_features)


In [108]:
tracker = LongTermReIDHungarianTracker(patience=20, obj_detect=obj_detect,reid_model  = reid_model )
res_seq2 = run_tracker_raw_seq(sequences=val_sequences, tracker=tracker)


Tracking: MOT16-02


100%|█████████████████████████████████████████| 600/600 [01:53<00:00,  5.27it/s]


Tracks found: 66
Runtime for MOT16-02: 114.2 s.
Tracking: MOT16-05


100%|█████████████████████████████████████████| 837/837 [01:20<00:00, 10.39it/s]


Tracks found: 114
Runtime for MOT16-05: 80.9 s.
Tracking: MOT16-09


100%|█████████████████████████████████████████| 525/525 [01:06<00:00,  7.90it/s]


Tracks found: 29
Runtime for MOT16-09: 66.6 s.
Tracking: MOT16-11


100%|█████████████████████████████████████████| 900/900 [02:04<00:00,  7.25it/s]


Tracks found: 71
Runtime for MOT16-11: 124.5 s.
Runtime for all sequences: 386.2 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP IDt IDa IDm
MOT16-02 36.9% 52.4% 28.4% 52.3% 96.2%  62  12  37 13  383  8870 218  220 49.0% 0.095 195  28  20
MOT16-05 55.4% 65.7% 48.0% 68.8% 94.2% 133  56  65 12  295  2158 121  154 62.8% 0.142 123  28  46
MOT16-09 50.4% 62.3% 42.3% 66.4% 97.8%  26  11  14  1   80  1789  50   81 64.0% 0.085  46  12   9
MOT16-11 62.8% 69.2% 57.4% 80.2% 96.6%  75  42  26  7  266  1868  64   88 76.7% 0.083  59  16  20
OVERALL  48.7% 61.2% 40.4% 63.5% 96.2% 296 121 142 33 1024 14685 453  543 59.9% 0.099 423  84  95
